In [ ]:

######## PARSING THE DATA #########
# we are cleaning up the dataset


In [ ]:
# disclaimer: you can't run all of the code in this notebook, this is just all of the code combined that we used in data preparation
# If you want to use all the code, make separate notebooks for each section separated by a comment cell

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/srv/scratch/ALL/DATA/NR-DBIND.csv', sep=';')

In [ ]:
df['ID'] = df['ID'].astype(int).astype(str)
df = df[df['p_binding_type'].isin(['pIC50', 'pKi'])] ## We only want p_binding_type to be pIC50 and pKi
df.head()

In [ ]:
# only choose the necessary columns
df = df[['ID', 'accession', 'smiles', 'CHEMBLID', 'p_binding_value']]
df.head()

In [ ]:
## 1. group by accession + smiles - all rows with the same accession and smiles are put into a group
## 2. aggregate with median - for each group take the median
## 3. reset the index to normal numbering
df = df.groupby(['accession', 'smiles']).agg('median').reset_index()
df.head()

In [ ]:
# only choose rows where p_binding_value is not NaN
df = df[df['p_binding_value'].notna()]

In [ ]:
# save df to csv
df.to_csv('parsed_data.csv', index=False)

In [ ]:

######## ADDING AMINO ACID SEQUENCES ######### (not as important as I had thought it would be)


In [ ]:
import pandas as pd
df = pd.read_csv('parsed_data.csv')

In [ ]:
a = df['accession'].unique()
for i in a:
    print(i)    #### this prints out the name of each protein in a new line ===> input to uniprot 


In [ ]:
######### after having gotten the fasta file from https://www.uniprot.org/uploadlists/   ===>
 
    
name = 'sequences.fasta' # I just named it that way

def getSeq(filename):                 #### makes a dictionary of the ID of the protein ('accession') as key and the sequence as value
    f = open(filename)
    id2seq = {}
    currkey = ""
    for line in f:
        if line.find(">") == 0:
            currkey = (line[1:].split("|")[1])
            id2seq[currkey] = ""
        else:            
            id2seq[currkey] = id2seq[currkey] + line.rstrip()
    f.close()
    return id2seq 
dictionary = getSeq(name)

df["AAsequence"] = df['accession'].map(dictionary)
df.to_csv('NEWDATASET.csv', index = 'false')

In [ ]:

######## PREPARING THE DATA FOR PROTEINS ######## (crucial)


In [ ]:

### ypu need the 3D structures of all of the proteins
### (nodes, edges, edge attributes, edge mapping)

import pickle
from typing import Optional, Tuple

import numpy as np
import pandas as pd
import torch
from pandas.core.frame import DataFrame
from torch_geometric.utils import to_undirected

aa_encoding = {
    "ala": 0,
    "arg": 1,
    "asn": 2,
    "asp": 3,
    "cys": 4,
    "gln": 5,
    "glu": 6,
    "gly": 7,
    "his": 8,
    "ile": 9,
    "leu": 10,
    "lys": 11,
    "met": 12,
    "phe": 13,
    "pro": 14,
    "ser": 15,
    "thr": 16,
    "trp": 17,
    "tyr": 18,
    "val": 19,
}

edge_type_encoding = {"cnt": 0, "combi": 1, "hbond": 2, "pept": 3, "ovl": 4}


def onehot_encode(position: int, count: Optional[int] = 20):
    """One-hot encode position
    Args:
        position (int): Which entry to set to 1
        count (Optional[int], optional): Max number of entries. Defaults to 20.
    Returns:
        [type]: [description]
    """
    t = [0] * count
    t[position] = 1
    return t


class ProteinEncoder:
    def __init__(self, config: dict):
        self.features = config["node_features"]
        self.edge_features = config["edge_features"]

    def encode_residue(self, residue: str) -> np.array:
        """Fully encode residue - one-hot and features

        Args:
            residue (str): One-letter residue name

        Returns:
            np.array: Concatenated features and one-hot encoding of residue name
        """
        if residue.lower() not in aa_encoding:
            return None
        elif self.features == "label":
            return aa_encoding[residue.lower()]
        elif self.features == "onehot":
            return onehot_encode(aa_encoding[residue.lower()])
        else:
            raise ValueError("Unknown features type!")

    def parse_sif(self, filename: str) -> Tuple[DataFrame, DataFrame]:
        """Parse a single sif file

        Args:
            filename (str): SIF file location

        Returns:
            Tuple[DataFrame, DataFrame]: nodes, edges DataFrames
        """
        nodes = []
        edges = []
        with open(filename, "r") as file:
            for line in file:
                line = line.strip()
                splitline = line.split()
                if len(splitline) != 3:
                    continue
                node1, edgetype, node2 = splitline
                node1split = node1.split(":")
                node2split = node2.split(":")
                if len(node1split) != 4:
                    continue
                if len(node2split) != 4:
                    continue
                chain1, resn1, x1, resaa1 = node1split
                chain2, resn2, x2, resaa2 = node2split
                if x1 != "_" or x2 != "_":
                    continue
                if resaa1.lower() not in aa_encoding or resaa2.lower() not in aa_encoding:
                    continue
                resn1 = int(resn1)
                resn2 = int(resn2)
                if resn1 == resn2:
                    continue
                edgesplit = edgetype.split(":")
                if len(edgesplit) != 2:
                    continue
                node1 = {"chain": chain1, "resn": resn1, "resaa": resaa1}
                node2 = {"chain": chain2, "resn": resn2, "resaa": resaa2}
                edgetype, _ = edgesplit
                edge = {
                    "resn1": resn1,
                    "resn2": resn2,
                    "type": edgetype,
                }
                nodes.append(node1)
                nodes.append(node2)
                edges.append(edge)
        nodes = pd.DataFrame(nodes).drop_duplicates()
        nodes = nodes.sort_values("resn").reset_index(drop=True).reset_index().set_index("resn")
        edges = pd.DataFrame(edges).drop_duplicates()
        node_idx = nodes["index"].to_dict()
        edges["node1"] = edges["resn1"].apply(lambda x: node_idx[x])
        edges["node2"] = edges["resn2"].apply(lambda x: node_idx[x])
        return nodes, edges

    def encode_nodes(self, nodes: pd.DataFrame) -> torch.Tensor:
        """Given dataframe of nodes create node features

        Args:
            nodes (pd.DataFrame): nodes dataframe from parse_sif

        Returns:
            torch.Tensor: Tensor of node features [n_nodes, *]
        """
        nodes.drop_duplicates(inplace=True)
        node_attr = [self.encode_residue(x) for x in nodes["resaa"]]
        node_attr = [x for x in node_attr if x is not None]
        node_attr = np.asarray(node_attr)
        if self.features == "label":
            node_attr = torch.tensor(node_attr, dtype=torch.long)
        else:
            node_attr = torch.tensor(node_attr, dtype=torch.float32)
        return node_attr

    def encode_edges(self, edges: pd.DataFrame) -> Tuple[torch.Tensor, torch.Tensor]:
        """Given dataframe of edges, create edge index and edge attributes

        Args:
            edges (pd.DataFrame): edges dataframe from parse_sif

        Returns:
            Tuple[torch.Tensor, torch.Tensor]: edge index [2,n_edges], edge attributes [n_edges, *]
        """
        edges.drop_duplicates(inplace=True)
        edge_index = edges[["node1", "node2"]].astype(int).values
        edge_index = torch.tensor(edge_index, dtype=torch.long)
        edge_index = edge_index.t().contiguous()
        if self.edge_features == "none":
            return edge_index, None
        edge_features = edges["type"].apply(lambda x: edge_type_encoding[x])
        if self.edge_features == "label":
            edge_features = torch.tensor(edge_features, dtype=torch.long)
            edge_index, edge_features = to_undirected(edge_index, edge_features)
            return edge_index, edge_features
        elif self.edge_features == "onehot":
            edge_features = edge_features.apply(onehot_encode, count=len(edge_type_encoding))
            edge_features = torch.tensor(edge_features, dtype=torch.float)
            edge_index, edge_features = to_undirected(edge_index, edge_features)
            return edge_index, edge_features

    def __call__(self, protein_sif: str) -> dict:
        """Fully process the protein

        Args:
            protein_sif (str): File location for sif file

        Returns:
            dict: standard format with x for node features, edge_index for edges etc
        """
        nodes, edges = self.parse_sif(protein_sif)
        node_attr = self.encode_nodes(nodes)
        edge_index, edge_attr = self.encode_edges(edges)
        return dict(x=node_attr, edge_index=edge_index, edge_attr=edge_attr, index_mapping=nodes["index"].to_dict())


def extract_name(protein_sif: str) -> str:
    """Extract the protein name from the sif filename"""
    return protein_sif.split("/")[-1].split("_")[0]


if __name__ == "__main__":
    proteins = pd.Series(list(snakemake.input.rins), name="sif")
    proteins = pd.DataFrame(proteins)
    proteins["ID"] = proteins["sif"].apply(extract_name)
    proteins.set_index("ID", inplace=True)
    prot_encoder = ProteinEncoder(snakemake.config)
    proteins["data"] = proteins["sif"].apply(prot_encoder)
    with open(snakemake.output.protein_pickle, "wb") as file:
        pickle.dump(proteins, file)

In [ ]:
f_pickled = pd.read_pickle('protein_data_label_label.pkl')
f_pickled.drop('sif', axis = 1)
def get_accession(name):
    name = name.split('/')[2]
    name = name.split('-')[1]
    return name
f_pickled['sif'] = f_pickled['sif'].apply(get_accession)
f_pickled['accession'] = f_pickled['sif']
f_pickled.drop('sif', inplace=True, axis=1)

In [ ]:
proteins = open('new_dataset_proteins.pckl', 'wb') 
pickle.dump(f_pickled, proteins)

In [ ]:

######## PREPARING THE DATA FOR DRUGS ######## (crucial)


In [ ]:
### (nodes, edges, edge attributes (bond types))
import pandas as pd                                       #### Importing some libraries, probably not all of them are important
import numpy as np
import torch
from rdkit import Chem
from rdkit.Chem import rdmolfiles, rdmolops
from torch_geometric.data import Data
from torch_geometric.utils import to_undirected
import numpy as np
import pickle 

dataset = pd.read_csv('NEWDATASET.csv')        ### opening the csv file where we have all our parsed data

def smiles_to_torch(smiles: str) -> Data:               #### this is a function that takes in the smiles of drug 
    '''                                                 #### (example:CC(C)c1onc(c1COc2ccc(cc2)c3ccc4c(cccc4c3)C(=O)O)c5c(Cl)cccc5Cl)
    Converts molecular smiles into torch data           #### it uses the torch library so i dont quite understand what its doing
    '''
    mol = Chem.MolFromSmiles(smiles)
    if not mol:  # when rdkit fails to read a molecule it returns None
      return np.nan
    new_order = rdmolfiles.CanonicalRankAtoms(mol)
    mol = rdmolops.RenumberAtoms(mol, new_order)
    dictionary = {'SINGLE':0,'DOUBLE':1,'AROMATIC':2}   ### dictionary for bond types
    edges = []
    edge_atributes = []
    for bond in mol.GetBonds():
      start, end = bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()
      bond_type = str(bond.GetBondType())                        ### the addition of information about bond type
      
      if bond_type in dictionary:
          type = dictionary[bond_type]
      else:
          type = 3
      edge_atributes.append(type)
      edges.append([start, end])

    if not edges:  # If no edges (bonds) were found, exit (single ion etc)
      return np.nan
    atom_features = []
    for atom in mol.GetAtoms():
      atom_num = atom.GetAtomicNum()
      atom_features.append(atom_num)

    x = torch.tensor(atom_features, dtype=torch.long)
    edge_index = torch.tensor(edges).t().contiguous()
    edge_atributes = torch.tensor(edge_atributes, dtype = torch.long)

    return dict(x=x, edge_index=edge_index, edge_atributes=edge_atributes)   #returns a dictionary of values (we are pairing the 
                                                                             #names of lists and list in tensor format of all nodes (atoms),
                                                                             #list of edges (bonds), and list of bond types
        
dataset['data'] = dataset['smiles'].apply(smiles_to_torch)           ### we apply the smiles_to_torch function to every smiles
dataset       ###  this just prints it out                           ### string, and putting into a new column called data

In [ ]:
drugs = open('new_dataset_drugs.pckl', 'wb') 
pickle.dump(dataset, drugs)

In [ ]:

######## FINAL: MAKING A LIST THAT WOULD BE FED TO THE MODEL ##########


In [ ]:
filehandler = open('../../../scratch/ALL/new_dataset_drugs.pckl', 'rb')                              ###
drug = pickle.load(filehandler)
filehandler = open('../../../DLDD/results/prepare_proteins/new_dataset_proteins.pckl', 'rb')         ### this is just where I saved them
prot = pickle.load(filehandler)

In [ ]:
prot_dict = {}
for i in range(len(prot)):
    row = prot.iloc[i]
    prot_dict[row['accession']] = row['data']

In [ ]:
list_for_model = []
for name, row in drug.iterrows():
    new_dict = {}
    accession = row['accession']
    data = row['data']
    new_dict['drug_x'] = data['x'] 
    new_dict['drug_edge_index'] = data['edge_index']
    new_dict['label'] = row['p_binding_value']
    new_dict['protein_x'] = prot_dict[accession]['x']
    new_dict['protein_edge_index'] = prot_dict[accession]['edge_index']
    list_for_model.append(new_dict)
list_for_model[0]

In [ ]:
final_list = open('final_list.pckl', 'wb') 
pickle.dump(list_for_model, final_list)                ### And finally we have a beautiful pickle file ready for the model
                                                       ### CONGRATS !!!